In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import sys;
sys.path.append(r'C:\Monitor\\')
sys.path.append(r'\\ac-geneva-24\E\grains trading\Streamlit\Monitor\\')

from datetime import datetime as dt
import numpy as np
import pandas as pd

import Utilities.Modeling as um
import Utilities.GLOBAL as GV
import Utilities.Utilities as uu
import statsmodels.api as sm

import Models.Soybean_USA_Yield_GA as sy

import warnings # supress warnings
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
scope = sy.Define_Scope()
raw_data = sy.Get_Data_All_Parallel(scope)

In [3]:
f='GA_soy_7'
i=142

r = uu.deserialize(f,comment=False)
m = r['model'][i]

r.keys()
print('Saved results:', np.mean(r['cv_p'][i]))

Saved results: 0.0013021448379618996


In [4]:
# Try replicating the saved results
wws = um.var_windows_from_cols(m.params.index)
model_df = um.extract_yearly_ww_variables(w_df = raw_data['w_w_df_all']['hist'],var_windows= wws)
model_df = pd.concat([raw_data['yield'], model_df], sort=True, axis=1, join='inner')

y_col  ='Yield'
y_df = model_df[[y_col]]


cols= [c for c in m.params.index if c != 'const']
X_df=model_df[cols]

folds = um.folds_expanding(model_df=model_df, min_train_size=10)

cv_score = um.stats_model_cross_validate(X_df, y_df, folds)

cv_score.keys()
print('Calculated results:', np.mean(cv_score['cv_p']))

Calculated results: 0.0013021448379618996


In [5]:
cv_score.keys()

dict_keys(['cv_models', 'cv_corr', 'cv_p', 'cv_r_sq', 'cv_y_test', 'cv_y_pred', 'cv_MAE', 'cv_MAPE'])

In [51]:
# [print(y.index[0],y['Yield'].iloc[0]) for y in cv_score['cv_y_test']] # good
# [print(y.index[0], y.iloc[0]) for y in cv_score['cv_y_pred']] # good
# [print(y.index[0]) for y in cv_score['cv_y_pred']] # good

# cv_score['cv_y_pred'].index[0]
# len(cv_score['cv_y_pred'])
# cv_score['cv_y_pred'][27].index[0]

cv_years = [y.index[0] for y in cv_score['cv_y_test']] 
cv_test = [y['Yield'].iloc[0] for y in cv_score['cv_y_test']]
cv_pred = [y.iloc[0] for y in cv_score['cv_y_pred']]

print(len(cv_years))
print(len(cv_test))
print(len(cv_pred))

27
27
27
